In [6]:
%pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\SSN\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [7]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
import mediapipe as mp


In [8]:
# Load your CSV file containing video labels and IDs
csv_path = 'features_df.csv'
df = pd.read_csv(csv_path)

# Get the unique classes
unique_classes = df['gloss'].nunique()

print("Number of unique classes:", unique_classes)

Number of unique classes: 2000


In [9]:
# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to extract keypoints from a frame


def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks,
                              mp_holistic.FACEMESH_CONTOURS)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks,
                              mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks,
                              mp_holistic.HAND_CONNECTIONS)  # Draw right hand connections


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 110, 10), thickness=1, circle_radius=1),
                              mp_drawing.DrawingSpec(
                                  color=(80, 256, 121), thickness=1, circle_radius=1)
                              )
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(80, 22, 10), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(80, 44, 121), thickness=2, circle_radius=2)
                              )
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(121, 22, 76), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(121, 44, 250), thickness=2, circle_radius=2)
                              )
    # Draw right hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(245, 117, 66), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(245, 66, 230), thickness=2, circle_radius=2)
                              )




In [14]:
from pymongo import MongoClient

# Set paths
csv_path = 'features_df.csv'
videos_folder = 'videos'


client = MongoClient('mongodb://localhost:27017/')
db = client['SLT']
collection = db['frames']



# Initialize Mediapipe
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Function to extract keypoints from a frame
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model.process(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return results

# Function to preprocess a video
def preprocess_video(video_path, target_size=(64, 64)):
    frames = []
    cap = cv2.VideoCapture(video_path)

    # Initialize Mediapipe model
    holistic = mp_holistic.Holistic()

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Extract keypoints using Mediapipe
        results = mediapipe_detection(frame, holistic)

        # Draw styled landmarks on the frame
        draw_styled_landmarks(frame, results)

        # Resize frame to the target size
        frame = cv2.resize(frame, target_size)
        frames.append(frame)
    
    frames = [cv2.resize(frame, target_size) for frame in frames]
    cap.release()
    return np.array(frames)

# Lists to store preprocessed data and labels
preprocessed_data = []
labels = []


start = 0
# Iterate through rows of the DataFrame

for index, row in df.iterrows():
    if start >= index:
        video_filename = str(row['video_id']) + '.mp4'
        sign_label = row['gloss']
        video_path = os.path.join(videos_folder, video_filename)

        # Check if the video file exists
        if os.path.exists(video_path):
            frames = preprocess_video(video_path)
            preprocessed_data.append(frames)
            data_entry = {
            'frames': frames.tolist(),  # Convert NumPy array to Python list
            'label': sign_label
            }
            collection.insert_one(data_entry)
            labels.append(sign_label)
            print("Processed Video:",sign_label)

client.close()

Processed Video: book


In [15]:
from pymongo import MongoClient
import numpy as np


# Set MongoDB connection details
client = MongoClient('mongodb://localhost:27017/')
db = client['SLT']
collection = db['frames']

# Retrieve data and labels from MongoDB
def retrieve_data_labels():
    data = []
    labels = []

    cursor = collection.find()
    for document in cursor:
        frames = np.array(document['frames'])
        label = document['label']
        data.append(frames)
        labels.append(label)

    return np.array(data), np.array(labels)

# Close the MongoDB connection


# Retrieve data and labels
X, y = retrieve_data_labels()

# Print the shape of the data and labels
print("Data shape:", X.shape)
print("Labels shape:", y.shape)

client.close()

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (63,) + inhomogeneous part.